In [23]:
import torch
import catdog
import torchvision
import torch.nn as nn
import torch.optim as optim

In [24]:
full_dataset = torchvision.datasets.ImageFolder("/home/amaury/datasets/PetImages", transform=catdog.Transform())

dataset_len = len(full_dataset)
train_size = int(0.8 * dataset_len)
test_size = dataset_len - train_size

trainset, testset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

batch_size = 4
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = catdog.Classes()

In [25]:
net = catdog.Model()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [26]:
for epoch in range(20):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:  # print every 2000 mini-batches
            print(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}")
            running_loss = 0.0

print("Finished Training")

[1,  2000] loss: 0.690
[1,  4000] loss: 0.655
[2,  2000] loss: 0.603
[2,  4000] loss: 0.578
[3,  2000] loss: 0.538
[3,  4000] loss: 0.533
[4,  2000] loss: 0.484
[4,  4000] loss: 0.493
[5,  2000] loss: 0.441
[5,  4000] loss: 0.448
[6,  2000] loss: 0.391
[6,  4000] loss: 0.401
[7,  2000] loss: 0.319
[7,  4000] loss: 0.347
[8,  2000] loss: 0.242
[8,  4000] loss: 0.282
[9,  2000] loss: 0.171
[9,  4000] loss: 0.215
[10,  2000] loss: 0.131
[10,  4000] loss: 0.166
[11,  2000] loss: 0.089
[11,  4000] loss: 0.128
[12,  2000] loss: 0.065
[12,  4000] loss: 0.105
[13,  2000] loss: 0.068
[13,  4000] loss: 0.080
[14,  2000] loss: 0.049
[14,  4000] loss: 0.067
[15,  2000] loss: 0.042
[15,  4000] loss: 0.059
[16,  2000] loss: 0.023
[16,  4000] loss: 0.040
[17,  2000] loss: 0.025
[17,  4000] loss: 0.037
[18,  2000] loss: 0.031
[18,  4000] loss: 0.028
[19,  2000] loss: 0.026
[19,  4000] loss: 0.038
[20,  2000] loss: 0.017
[20,  4000] loss: 0.015
Finished Training


In [27]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy of the network on the {total} test images: {100 * correct // total} %")

Accuracy of the network on the 5000 test images: 74 %


In [28]:
PATH = "./cat_dog.pth"
torch.save(net, PATH)